Agente de finanças

In [5]:
import os
from dotenv import load_dotenv
import psycopg2
from langchain.llms import OpenAI

load_dotenv()

True

In [6]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [7]:
def conectar_db():
    try:
        conn = psycopg2.connect(
            dbname=os.getenv("DB_NAME"),
            user=os.getenv("DB_USER"),
            password=os.getenv("DB_PASSWORD"),
            host='localhost',
            port=os.getenv("DB_PORT")
        )
        print("Conexão ao banco de dados PostgreSQL bem-sucedida!")
        return conn
    except psycopg2.OperationalError as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        raise

In [8]:
def obter_dados_acoes(pergunta):
    conn = conectar_db()
    cur = conn.cursor()
    query = """
    SELECT s.ticker, s.name, sp.date, sp.open, sp.high, sp.low, sp.close, sp.volume
    FROM stocks s
    JOIN stock_prices sp ON s.id = sp.stock_id
    WHERE s.name ILIKE %s OR s.ticker ILIKE %s;
    """
    cur.execute(query, (f"%{pergunta}%", f"%{pergunta}%"))
    resultados = cur.fetchall()
    cur.close()
    conn.close()
    return resultados

In [9]:
llm = OpenAI(temperature=0.6, api_key=os.getenv("OPENAI_API_KEY"))

/var/folders/8_/k1fm88l93q978ncwnjpl0rzr0000gn/T/ipykernel_31860/1523739255.py:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  llm = OpenAI(temperature=0.6, api_key=os.getenv("OPENAI_API_KEY"))


In [11]:
def analisar_dados_acoes(dados_acoes): 
 
    prompt = "Analise os seguintes dados de ações e forneça insights detalhados:\n"
    
    for dado in dados_acoes:
        prompt += (f"Ticker: {dado[0]}, Nome: {dado[1]}, Data: {dado[2]}, "
                   f"Abertura: {dado[3]}, Máxima: {dado[4]}, Mínima: {dado[5]}, "
                   f"Fechamento: {dado[6]}, Volume: {dado[7]}\n")
 
    prompt += "\nPor favor, forneça um resumo e quaisquer tendências ou insights notáveis."
    
    insights = llm(prompt)
    return insights

def responder_pergunta(pergunta):
    dados_acoes = obter_dados_acoes(pergunta)
    if dados_acoes:
        resposta = f"Dados encontrados para '{pergunta}':\n"
        for dado in dados_acoes:
            resposta += (f"Ticker: {dado[0]}, Nome: {dado[1]}, Data: {dado[2]}, "
                         f"Abertura: {dado[3]}, Máxima: {dado[4]}, Mínima: {dado[5]}, "
                         f"Fechamento: {dado[6]}, Volume: {dado[7]}\n")
        resposta += "\nInsights da OpenAI:\n"
        resposta += analisar_dados_acoes(dados_acoes)
    else:
        resposta = f"Nenhum dado encontrado para '{pergunta}'."
    return resposta

In [12]:
def main():
    pergunta = input("Sobre qual ação você deseja saber informações? ")
    resposta = responder_pergunta(pergunta)
    print(resposta)

if __name__ == "__main__":
    main()

Conexão ao banco de dados PostgreSQL bem-sucedida!


/var/folders/8_/k1fm88l93q978ncwnjpl0rzr0000gn/T/ipykernel_31860/552221584.py:12: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  insights = llm(prompt)


Dados encontrados para 'AAPL':
Ticker: AAPL, Nome: Apple Inc., Data: 2023-08-01, Abertura: 145.00, Máxima: 147.50, Mínima: 144.00, Fechamento: 146.50, Volume: 80000000
Ticker: AAPL, Nome: Apple Inc., Data: 2023-08-02, Abertura: 146.00, Máxima: 148.00, Mínima: 145.50, Fechamento: 147.00, Volume: 82000000
Ticker: AAPL, Nome: Apple Inc., Data: 2023-08-03, Abertura: 147.50, Máxima: 149.00, Mínima: 146.00, Fechamento: 148.50, Volume: 85000000

Insights da OpenAI:


O ticker AAPL representa as ações da empresa Apple Inc. e os dados fornecidos são referentes aos dias 1, 2 e 3 de agosto de 2023. Podemos observar que houve uma tendência de alta nos preços das ações durante esses três dias.

No dia 1, as ações da Apple abriram com um valor de 145.00 e atingiram sua máxima em 147.50, fechando o dia em 146.50. O volume de negociações foi de 80000000. Já no dia 2, houve uma pequena variação de preço, com a abertura em 146.00 e a máxima em 148.00. As ações fecharam o dia em 147.00 e o volume de nego